In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch

In [12]:
base_dir = "/media/davidclark/DATA/VectorizedNets/"
run_dirs = os.listdir(base_dir)
num_runs = len(run_dirs)

#CIFAR/MNIST
#FC/CONV/LC
#Vec/Nonvec
#DF/BP
#Mixed/Mono
#Train/Test
#Runs
acc_vals = np.zeros((
    2, 3, 2, 2, 2, 2, num_runs
))

num_runs

5

In [13]:
dsets = ['mnist', 'cifar']
archs = ['fc', 'conv', 'lc']
styles = ['vec', 'nonvec']
algos = ['df', 'bp']
signs = ['mono', 'mixed']

def build_dir_name(dset_idx, arch_idx, style_idx, algo_idx, sign_idx, run_idx):
    name = 'models_{}/{}_{}_{}_{}_{}'.format(run_idx,
        dsets[dset_idx], styles[style_idx], archs[arch_idx], algos[algo_idx], signs[sign_idx])
    return name
    

for dset_idx in range(len(dsets)):
    for arch_idx in range(len(archs)):
        for style_idx in range(len(styles)):
            for algo_idx in range(len(algos)):
                for sign_idx in range(len(signs)):
                    for run_idx in range(num_runs):
                        subdir_name = build_dir_name(dset_idx, arch_idx, style_idx, algo_idx, sign_idx, run_idx)
                        dir_name = base_dir + subdir_name
                        #print(dir_name)
                        if not os.path.isdir(dir_name):
                            acc_vals[dset_idx, arch_idx, style_idx, algo_idx, sign_idx, :, run_idx] = np.nan
                            continue
                        epoch_dirs = os.listdir(dir_name)
                        epoch_indices = [int(name.split('.')[0].split('_')[1]) for name in epoch_dirs]
                        max_epoch = np.max(epoch_indices)
                        to_load = dir_name + '/epoch_{}.pt'.format(max_epoch)
                        checkpoint = torch.load(to_load, map_location='cpu')
                        train_acc = checkpoint['train_accuracy']
                        if max_epoch < 190 and train_acc < 0.999999:
                            print(dir_name, max_epoch, train_acc)
                            acc_vals[dset_idx, arch_idx, style_idx, algo_idx, sign_idx, :, run_idx] = np.nan
                            continue
                        test_acc = checkpoint['test_accuracy']
                        acc_vals[dset_idx, arch_idx, style_idx, algo_idx, sign_idx, :, run_idx] = [train_acc, test_acc]



In [14]:
err_vals = 100*(1 - acc_vals)
err_vals_mean = np.nanmean(err_vals, axis=-1)
err_vals_std = np.nanstd(err_vals, axis=-1)


In [25]:
err_vals[1, 1, 1, 0, 1]

array([[16.744, 12.898, 19.188, 25.07 , 12.326],
       [43.52 , 43.86 , 43.96 , 45.71 , 43.31 ]])

In [24]:
archs_short = ['FC', "Conv", "LC"]
styles_short = ["-Vec", ""]

for dset_idx in range(len(dsets)):
    print("& DF+ & BP+ & DF & BP \\\\ \\hline")
    for arch_idx in range(len(archs)):
        for style_idx in range(len(styles)):
            print("{}{} & ".format(archs_short[arch_idx], styles_short[style_idx]), end="")
            for sign_idx in range(len(signs)):
                for algo_idx in range(len(algos)):    
                    train, test = err_vals_mean[dset_idx, arch_idx, style_idx, algo_idx, sign_idx]
                    train, test = train.round(2), test.round(2)
                    
                    train_std, test_std = err_vals_std[dset_idx, arch_idx, style_idx, algo_idx, sign_idx]
                    train_std, test_std = train_std.round(2), test_std.round(2)
                    
                    #disp = '{} $\pm$ {}'.format(test, test_std)
                    disp = '{}'.format(test)
                    if train > 0:
                        #disp = disp + ' ({} $\pm$ {})'.format(train, train_std)
                        disp = disp + ' ({})'.format(train)
                    if style_idx == 0 and sign_idx == 0 and algo_idx == 0:
                        disp = '\hglht{' + disp + '}'
                    print(disp, end="")
                    if not (algo_idx == 1 and sign_idx == 1):
                        print(" & ", end="")
            print(" \\\\", end="")
            print("")
    print("\\hline\n\n", end="")
                    

& DF+ & BP+ & DF & BP \\ \hline
FC-Vec & \hglht{1.87 (0.06)} & 1.93 (0.05) & 2.32 (0.33) & 1.84 (0.07) \\
FC & 2.2 (0.3) & 1.36 & 2.09 (0.19) & 1.29 \\
Conv-Vec & \hglht{2.33 (1.03)} & 1.3 (0.19) & 1.83 (0.83) & 0.8 \\
Conv & 1.56 (0.67) & 0.71 & 1.64 (0.42) & 0.65 \\
LC-Vec & \hglht{1.78} & 1.64 & 1.84 (0.05) & 1.44 \\
LC & 1.98 (0.32) & 1.21 & 1.48 (0.09) & 1.07 \\
\hline

& DF+ & BP+ & DF & BP \\ \hline
FC-Vec & \hglht{47.62 (1.25)} & 47.03 (0.72) & 48.86 (2.02) & 45.98 (0.78) \\
FC & 48.69 (3.28) & 45.42 (0.96) & 49.54 (2.88) & 45.69 (0.73) \\
Conv-Vec & \hglht{33.74 (28.83)} & 30.85 (16.29) & 38.43 (20.59) & 30.54 (1.71) \\
Conv & 54.18 (48.43) & 32.13 (0.23) & 44.07 (17.25) & 28.8 (0.15) \\
LC-Vec & \hglht{41.08 (0.83)} & 41.01 (0.34) & 40.11 (0.83) & 38.77 (0.36) \\
LC & 41.18 (10.62) & 35.51 & 39.41 (2.94) & 32.32 \\
\hline



In [40]:
np.array([0.0051]).round(2)

array([0.01])

In [28]:
a.round(2)

0.5